In [27]:
import sys
import os
import sympy as sp
import numpy as np

# Add the correct path to the P2 folder
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

from gauss_fehlerfortpflanzung import evaluate_gaussian_error
from geraden_fit import geraden_fit
from scientific_error_rounder import runden_und_speichern
from mean_calc_w_err import err_weighted_mean

In [25]:
# Mittelwerte bilden:
import numpy as np

def mean_calc(z_input, err_input, goal='data weighting'):
    """
    Berechnet den gewichteten Mittelwert eines Wertearrays unter Berücksichtigung individueller Fehlerwerte.
    """
    valid_indices = [i for i in range(len(z_input)) if z_input[i] is not None and err_input[i] is not None]

    if not valid_indices:  # Keine gültigen Werte
        return None

    if len(valid_indices) == 1:  # Nur ein gültiger Wert
        value = z_input[valid_indices[0]]
        error = err_input[valid_indices[0]]
        return value if goal == 'data weighting' else error

    z_input = np.array([z_input[i] for i in valid_indices])
    err_input = np.array([err_input[i] for i in valid_indices])

    mean_1 = np.sum(z_input / err_input ** 2)
    mean_2 = np.sum(1 / err_input ** 2)

    if mean_2 == 0:  # Schutz gegen Division durch Null
        return None

    if goal == 'data weighting':
        return mean_1 / mean_2
    elif goal == 'error':
        return np.sqrt(1 / mean_2)


def process_file(input_file, output_file):
    # Datei einlesen
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # Überspringe die erste Zeile (Spaltennamen)
    data = [line.split() for line in lines[1:]]

    # Debugging: Eingabedaten prüfen
    print("Eingabedaten:")
    for line in data[:5]:  # Nur die ersten 5 Zeilen anzeigen
        print(line)

    # Konvertiere Werte zu float oder None
    parsed_data = [[float(val) if val != 'None' else None for val in row] for row in data]

    # Debugging: Konvertierte Daten prüfen
    print("\nKonvertierte Daten (erste Zeilen):")
    for row in parsed_data[:5]:
        print(row)

    # Extrahiere Werte und Fehler
    n_values = 5  # Anzahl der Werte pro Kategorie
    categories = ['t_up', 't_mid', 't_down']
    results = []

    for row_idx, row in enumerate(parsed_data):
        row_results = []
        for i, category in enumerate(categories):
            # Werte und Fehler korrekt extrahieren
            values = [row[j] for j in range(i * 10, i * 10 + 10, 2)]
            errors = [row[j] for j in range(i * 10 + 1, i * 10 + 10, 2)]

            # Debugging: Überprüfen, welche Werte extrahiert wurden
            print(f"\nZeile {row_idx + 1}, Kategorie {categories[i]}:")
            print(f"Werte: {values}")
            print(f"Fehler: {errors}")

            # Berechnung des gewichteten Mittels und des Fehlers
            mean_val = mean_calc(values, errors, goal='data weighting')
            mean_err = mean_calc(values, errors, goal='error')

            # Debugging: Ergebnisse der Berechnungen prüfen
            print(f"Berechnetes Mittel: {mean_val}")
            print(f"Berechneter Fehler: {mean_err}")

            row_results.extend([mean_val, mean_err])
        results.append(row_results)

    # Speichern der Ergebnisse in eine txt-Datei
    with open(output_file, 'w') as file:
        # Spaltennamen schreiben
        col_names = [f"{cat}_mean\t{cat}_err" for cat in categories]
        file.write("\t".join(col_names) + "\n")

        # Ergebnisse Zeile für Zeile schreiben
        for result in results:
            line = "\t".join(f"{val:.4f}" if val is not None else "None" for val in result)
            file.write(line + "\n")

    print("\nErgebnisse erfolgreich geschrieben in:", output_file)


# Beispiel: Datei verarbeiten
input_file = 'c.txt'  # Name der Eingabedatei
output_file = 'c_mean.txt'  # Name der Ausgabedatei
process_file(input_file, output_file)


Eingabedaten:
['5.34', '0.25', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1.74', '0.25', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1.61', '0.25', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']
['3.28', '0.25', '3.1', '0.25', 'None', 'None', 'None', 'None', 'None', 'None', '2.95', '0.25', '2.95', '0.25', 'None', 'None', 'None', 'None', 'None', 'None', '0.84', '0.25', '0.91', '0.25', 'None', 'None', 'None', 'None', 'None', 'None']
['17.31', '0.25', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '2.15', '0.25', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1.98', '0.25', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']
['2.48', '0.25', '2.87', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1.04', '0.25', '0.62', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '0.44', '0.25', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']
['0.38', '0.25', 'None

In [ ]:
### berechne v ###
# 5Kacheln = 0.5mm = 0.0005m
d = 0.0005
var_names=[t_up_mean, t_mid_mean,	t_down_mean]=sp.symbols('t_up_mean	t_mid_mean	t_down_mean')
formula_list=[
    d/t_up_mean, #v_up
    d/t_mid_mean, #v_0
    d/t_down_mean, #v_down
] 
evaluate_gaussian_error('c_mean.txt',formulas=formula_list, variables=var_names, result_length=10,result_names=['v_up', 'v_0', 'v_down'], output_file_suffix= 'v')



sqrt((-0.0005*del_t_up_mean/t_up_mean**2)**2 + (0)**2 + (0)**2)
sqrt((0)**2 + (-0.0005*del_t_mid_mean/t_mid_mean**2)**2 + (0)**2)
sqrt((0)**2 + (0)**2 + (-0.0005*del_t_down_mean/t_down_mean**2)**2)
Zeile 1: [9.3633e-05, 4.3836e-06, 0.0002873563, 4.12868e-05, 0.000310559, 4.82234e-05]
Zeile 2: [0.0001567398, 8.687e-06, 0.0001694915, 1.0158e-05, 0.0005714286, 0.0001154612]
Zeile 3: [2.8885e-05, 4.172e-07, 0.0002325581, 2.70416e-05, 0.0002525253, 3.18845e-05]
Zeile 4: [0.0002016129, 2.03239e-05, 0.0004807692, 0.0001155695, 0.0011363636, 0.0006456612]
Zeile 5: [0.0013157895, 0.000865651, 0.0003787879, 7.17401e-05, 0.0026315789, 0.0034626039]
Zeile 6: [0.0001096491, 6.0115e-06, 0.0001445087, 1.04414e-05, 0.0003472222, 6.02816e-05]
Zeile 7: [0.0009433962, 0.000256853, 0.000245098, 1.73371e-05, 0.001369863, 0.0006635391]
Zeile 8: [5.02008e-05, 1.2601e-06, 5.37057e-05, 1.4422e-06, 0.0005154639, 0.0001328515]
Zeile 9: [0.0007407407, 0.0001940192, 0.0003424658, 4.14712e-05, 0.0012195122, 0.00052

In [34]:
runden_und_speichern('c_mean.txt')
runden_und_speichern('c_mean_v.txt')

Die gerundeten Daten wurden in der Datei "c_mean_rounded.txt" gespeichert.
Die gerundeten Daten wurden in der Datei "c_mean_v_rounded.txt" gespeichert.


In [ ]:
var_names=[v_up_mean, v_mid_mean, v_down_mean]=sp.symbols('v_up_mean	v_mid_mean	v_down_mean')
formula_list=[
    2*v_mid_mean - v_down_mean + v_up_mean
] # spezifische Ladung e/m
evaluate_gaussian_error('c_mean_v.txt',formulas=formula_list, variables=var_names, result_length=10,result_names=['2*v_0-(v_down-v_up)'], output_file_suffix= 'control')

# -> Da die Unsicherheit fuer Messung 1, 3, 4, 8, 13, 15, 17 da die 

sqrt((del_v_up_mean)**2 + (2*del_v_mid_mean)**2 + (-del_v_down_mean)**2)
Zeile 1: [0.0003577866, 9.57241e-05]
Zeile 2: [-7.57058e-05, 0.0001175563]
Zeile 3: [0.0002414759, 6.27837e-05]
Zeile 4: [2.67877e-05, 0.000686088]
Zeile 5: [-0.0005582136, 0.0035720532]
Zeile 6: [5.14443e-05, 6.40789e-05]
Zeile 7: [6.37292e-05, 0.0007123622]
Zeile 8: [-0.0003578517, 0.0001328888]
Zeile 9: [0.0002061601, 0.0005666304]
Zeile 10: [-0.0001009256, 0.0007977195]
Zeile 11: [4.27703e-05, 0.0002833362]
Zeile 12: [-0.0003682803, 0.0010462826]
Zeile 13: [0.0002736094, 0.0002257516]
Zeile 14: [5.1384e-06, 1.75842e-05]
Zeile 15: [-1.48561e-05, 6.3298e-06]
Zeile 16: [-2.4091e-06, 4.7749e-06]
Zeile 17: [7.8489e-06, 1.48341e-05]
Auswertung abgeschlossen. Ergebnisse wurden in 'c_mean_v_control.txt' gespeichert.
